# Demonstation of an application of the Fast-Fourier-Transform

This demo shows an examplary application of the [Fast-Fourier-Transform (FFT)](https://en.wikipedia.org/wiki/Fast_Fourier_transform), which is an efficient way to implement the [Discrete Fourier Transform (DFT)](https://en.wikipedia.org/wiki/Discrete_Fourier_transform). 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on signals and systems hold at the [university of applied siences, Berlin](https://www.htw-berlin.de/).

**Import libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

**Why FFT???**

??? Some interesting effects to investigate:

* detect (also weak) frequencies
* logarithmic scaling of spectrum (for low 2nd amplitude)
* non-integer oscillations in observation period --> non periodic time signal (scalloping loss)
* windowing in time domain with different window functions ([spectral leakage](https://en.wikipedia.org/wiki/Spectral_leakage))

...

**Actual visualization**

In [2]:

def plot_signals(f1=2.0, f2=40.0, a2=0.1, scale='lin', window='Rectangular'):

    # sample rate
    sr = 1000
    # duration of the signal
    dur = 1
    # time and frequency axis
    t = np.arange(0, dur, 1 / sr)
    f = np.fft.fftshift(np.fft.fftfreq(t.size, 1 / sr))

    # generate signals
    sig1 = np.cos(f1 * 2 * np.pi * t + np.pi/10)
    sig2 = a2 * np.cos(f2 * 2 * np.pi * t)
    sig = sig1 + sig2

    # windowing?
    if window == 'Rectangular':
        w = signal.windows.boxcar(sig.size)
        sig *= w
    elif window == 'Hanning':
        w = signal.windows.hann(sig.size)
        sig *= w
    elif window == 'Blackman-Harris':
        w = signal.windows.blackmanharris(sig.size)
        sig *= w

    # amplitude spectrum
    Sig = np.abs(np.fft.fftshift(np.fft.fft(sig)) / sig.size)

    # lin / log scaling of spectrum
    if scale == 'log':
        np.seterr(divide = 'ignore')
        Sig = 20 * np.log10(Sig)
        ylabel = r'amplitude / 1V in dB'
        ylim = (-150, 0)
    else:
        ylabel = 'amplitude'
        ylim = (0, 0.6)

    # ploting stuff
    n_row = 1
    n_col = 2
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)

    ax = fig.add_subplot(n_row, n_col, 1)
    ax.plot(t, sig)
    ax.plot(t, w, color=(0.5,0.5,0.5), ls='dotted')
    ax.set(xlabel='time / s', ylabel='amplitude', xlim=(0, dur))

    ax = fig.add_subplot(n_row, n_col, 2)
    ax.plot(f, Sig)
    ax.set(xlabel='frequency / Hz', ylabel=ylabel, xlim=(-100, 100), ylim=ylim)

    plt.tight_layout()
    plt.show()

# sliders
style = {'description_width': '20%'}
w_scale = widgets.RadioButtons(options=['lin', 'log'], value='lin', description='Scale:', disabled=False)
w_f1 = widgets.FloatSlider(min=0.5, max=10.0, step=0.5, value=2.0, continuous_update=False, description=r'Freq. 1:', style=style)
w_f2 = widgets.IntSlider(min=10, max=100, step=1, value=40, continuous_update=False, description=r'Freq. 2:', style=style)
w_a2 = widgets.Dropdown(options=[('1',1.0), ('1/10',0.1), ('1/100',0.01), ('1/1000',0.001), ('1/10000',0.0001)], value=1.0, description='Amp. 2:', disabled=False, style=style)
w_window = widgets.Dropdown(options=['Rectangular', 'Hanning', 'Blackman-Harris'], value='Rectangular', description='Window:', disabled=False, style=style)

ui_u = widgets.HBox([w_f1, w_f2, w_a2])
ui_l = widgets.HBox([w_scale, w_window])
ui = widgets.VBox([ui_u, ui_l])

out = widgets.interactive_output(plot_signals, {'scale':w_scale, 'f1':w_f1, 'f2':w_f2, 'a2':w_a2, 'window':w_window})
out.layout.height = '350px'

display(ui, out)

Output(layout=Layout(height='350px'))